<a href="https://colab.research.google.com/github/khanhvy31/humanactivity_wearablesensor/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class modelV1(nn.Module):
  def __init__(self, config):
    super(modelV1, self).__init__()
    self.lstm = nn.LSTM(config.feature_length, 50, 1, batch_first=True)
    ffn_input_shape = self.computeShape(config.batch_size, config.sample_length)
    self.head = nn.Linear(ffn_input_shape, 13)

  def forward(self, x):
    x, (c, h) = self.lstm(x)
    # print(x.shape)
    x = torch.flatten(x, start_dim=1)
    # print(x.shape)
    x = self.head(x)
    return x

  def computeShape(self, batch_size, sample_length=8):
    'compute the flattened shape to FFN head'
    tens = torch.randn((batch_size, sample_length, config.feature_length))
    x, (c, h) = self.lstm(tens)
    return torch.flatten(x, start_dim=1).shape[1]

In [ ]:
class ConvNet1D(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv1d(23, 64, kernel_size=3),
            nn.ReLU(),
            nn.Dropout(0.5))
        self.layer2 = nn.Flatten()
        
        ffn_input_shape = self.computeShape(config.batch_size, config.sample_length)

        self.layer3 = nn.Sequential(
            nn.Linear(ffn_input_shape,128),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Linear(128,13),
            nn.Softmax())
        
        self.c = config

    def forward(self, x):
        x = x.permute((0, 2, 1))
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        return out

    def computeShape(self, batch_size, sample_length=8):
      'compute the flattened shape to FFN head'
      tens = torch.randn((batch_size, 23, sample_length))
      x = self.layer1(tens)
      x = self.layer2(x)
      return x.shape[1]

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers, output_size):
        super().__init__()

        # Defining some parameters
        self.input_size = (8,23)
        self.hidden_dim = 13
        self.n_layers = 1
        self.output_size = (23,13)

        #Defining the layers
        # RNN Layer
        self.rnn = torch.nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        h = self.rnn(x)[0]
        x = self.fc(h)
        return x